# Script to install required libraries and extract text from various file formats

In [ ]:
# Install required libraries
import os
import sys
import subprocess

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = [
    "pdfplumber==0.5.28",
    "pytesseract==0.3.8",
    "Pillow==9.2.0",
    "python-docx==0.8.11",
    "pandas==1.4.2",
    "openpyxl==3.0.9",
    "xlrd==2.0.1"
]

for package in required_packages:
    install(package)

# Ensure Tesseract-OCR is installed
try:
    pytesseract.get_tesseract_version()
except (FileNotFoundError, pytesseract.pytesseract.TesseractNotFoundError):
    print("Tesseract-OCR is not installed. Please install it from: https://github.com/tesseract-ocr/tesseract")

In [ ]:
# Import the installed libraries
import os
import sys
import subprocess
import pdfplumber
import pytesseract
from PIL import Image
from docx import Document
import pandas as pd

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()  # Extract text from the page
            if page_text:  # Check if the extracted text is not None
                text += page_text  # Only concatenate if page_text is not None
    return text

def extract_text_from_image(file_path):
    text = pytesseract.image_to_string(Image.open(file_path))
    return text

def extract_text_from_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def extract_text_from_excel(file_path):
    df = pd.read_excel(file_path, engine='openpyxl')
    text = df.to_string()
    return text

def extract_text_from_csv(file_path):
    df = pd.read_csv(file_path)
    text = df.to_string()
    return text

def extract_text_from_files(directory_path):
    all_texts = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if file_name.endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
        elif file_name.endswith('.jpg') or file_name.endswith('.png'):
            text = extract_text_from_image(file_path)
        elif file_name.endswith('.docx'):
            text = extract_text_from_docx(file_path)
        elif file_name.endswith('.xls') or file_name.endswith('.xlsx'):
            text = extract_text_from_excel(file_path)
        elif file_name.endswith('.csv'):
            text = extract_text_from_csv(file_path)
        else:
            with open(file_path, 'r') as file:
                text = file.read()
        all_texts.append(text)
    return all_texts

In [ ]:
# Example usage:
directory_path = ''
texts = extract_text_from_files(directory_path)
print(texts)

# Code pour analyser un corpus de texte et créer un CSV de Journal comptable selon le plan comptable général (PCG) de Madagascar

Importer les bibliothèques nécessaires

In [ ]:
!pip install spacy
!python -m spacy download fr_core_news_sm

In [ ]:
import re
import pandas as pd
import datetime
import spacy

Charger le modèle de NLP pour analyser le contexte et les entités nommés

In [ ]:
# Charger le modèle de langue française de spaCy
nlp = spacy.load('fr_core_news_sm')

Pré-traitement du corpus de texte

In [ ]:
# Fonction pour convertir les nombres français en entiers
def french_number_to_int(text):
    num_dict = {
        "un": 1, "deux": 2, "trois": 3, "quatre": 4, "cinq": 5, "six": 6, "sept": 7, "huit": 8, "neuf": 9, "dix": 10,
        "onze": 11, "douze": 12, "treize": 13, "quatorze": 14, "quinze": 15, "seize": 16, "dix-sept": 17, "dix-huit": 18, "dix-neuf": 19,
        "vingt": 20, "trente": 30, "quarante": 40, "cinquante": 50, "soixante": 60, "soixante-dix": 70, "quatre-vingt": 80, "quatre-vingt-dix": 90,
        "cent": 100, "mille": 1000, "million": 1000000
    }
    text = text.lower().replace("-", " ").replace(",", "")
    words = text.split()
    total = 0
    temp = 0
    for word in words:
        if word in num_dict:
            temp += num_dict[word]
        elif word == "mille":
            temp *= 1000
            total += temp
            temp = 0
        elif word == "million":
            temp *= 1000000
            total += temp
            temp = 0
    total += temp
    return total

# Fonction pour extraire les montants du texte
def extract_amounts(text):
    amounts = []
    patterns = [
        re.compile(r"somme de (.*?) (Ariary|Ar|Euro|Dollar)", re.IGNORECASE),
        re.compile(r"montant[:\s]*([\d\s,.]+) (Ariary|Ar|Euro|Dollar)", re.IGNORECASE),
        re.compile(r"([\d\s,.]+) (Ariary|Ar|Euro|Dollar)", re.IGNORECASE)
    ]
    for pattern in patterns:
        matches = pattern.findall(text)
        for match in matches:
            clean_match = match[0].strip().replace(",", "").replace(" ", "")
            try:
                amount = int(clean_match)
                amounts.append(amount)
            except ValueError:
                pass
    return amounts

Analyse comptable

In [ ]:
# Déterminer le compte crédité en fonction du compte débité selon le PCG de Madagascar
def get_credit_account(debit_account):
    account_mapping = {
        "2": 404,
        "60": 401,
        "61": 401,
        "621": 401,
        "622": 401,
        "623": 401,
        "624": 401,
        "625": 467,
        "626": 401,
        "627": 401,
        "628": 471,
        "63": 44,
        "64": 421,
        "65": 471,
        "66": 471,
        "67": 471,
        "68": 28,
        "601": 401,
        "602": 401,
        "603": 401,
        "604": 401,
        "605": 401,
        "606": 401,
        "607": 401,
        "608": 401,
        "609": 401,
        "615": 401,
        "616": 401,
        "618": 401,
        "620": 401
    }
    return account_mapping.get(str(debit_account), 0)

# Déterminer le libellé de la transaction en fonction de son type et du contexte
def determine_libelle(text):
    if "honoraire" in text.lower():
        return "Honoraire - " + extract_context(text)
    elif "achat" in text.lower() or "fourniture" in text.lower():
        return "Achat - " + extract_context(text)
    elif "loyer" in text.lower():
        return "Loyer - " + extract_context(text)
    elif "service" in text.lower():
        return "Service - " + extract_context(text)
    else:
        return "Autre - " + extract_context(text)

# Extraire des informations contextuelles pour enrichir le libellé
def extract_context(text):
    context = ""
    # Rechercher des termes spécifiques pour le contexte
    if "ordinateur" in text.lower():
        context = "Ordinateur"
    elif "fourniture de bureau" in text.lower():
        context = "Fourniture de bureau"
    elif "transport" in text.lower():
        context = "Transport"
    elif "formation" in text.lower():
        context = "Formation"
    elif "entretien" in text.lower():
        context = "Entretien"
    elif "publicité" in text.lower():
        context = "Publicité"
    elif "consultant" in text.lower():
        context = "Consultant"
    elif "bureau" in text.lower():
        context = "Fournitures de bureau"
    elif "logiciel" in text.lower():
        context = "Logiciel"
    elif "maintenance" in text.lower():
        context = "Maintenance"
    elif "électricité" in text.lower():
        context = "Electricité"
    elif "eau" in text.lower():
        context = "Eau"
    elif "internet" in text.lower():
        context = "Internet"
    else:
        # Prendre les premiers mots comme contexte par défaut
        words = text.split()
        context = " ".join(words[:5]) + "..."
    return context

# Déterminer le compte débité en fonction du libellé et du texte selon le PCG de Madagascar
def get_debit_account(libelle, text):
    libelle_to_account = {
        "Honoraire": "621",  # Compte pour Honoraire
        "Achat": "60",       # Compte pour Achat
        "Loyer": "624",      # Compte pour Loyer
        "Service": "628",    # Compte pour Service
        "Autre": "62"        # Compte par défaut pour Autre
    }
    
    # Ajustement pour des cas spécifiques basés sur le texte
    if "fourniture" in text.lower():
        return "606"  # Compte pour Fournitures
    if "transport" in text.lower():
        return "625"  # Compte pour Transport
    if "formation" in text.lower():
        return "623"  # Compte pour Formation
    if "entretien" in text.lower():
        return "615"  # Compte pour Entretien
    if "publicité" in text.lower():
        return "623"  # Compte pour Publicité
    if "consultant" in text.lower():
        return "622"  # Compte pour Consultant
    if "bureau" in text.lower():
        return "606"  # Compte pour Fournitures de bureau
    if "ordinateur" in text.lower():
        return "606"  # Compte pour Achat d'ordinateur
    if "logiciel" in text.lower():
        return "606"  # Compte pour Achat de logiciel
    if "service informatique" in text.lower():
        return "628"  # Compte pour Service informatique
    if "maintenance" in text.lower():
        return "615"  # Compte pour Maintenance
    if "électricité" in text.lower():
        return "606"  # Compte pour Electricité
    if "eau" in text.lower():
        return "606"  # Compte pour Eau
    if "internet" in text.lower():
        return "626"  # Compte pour Internet
    if "téléphone" in text.lower():
        return "626"  # Compte pour Téléphone
    if "location" in text.lower():
        return "621"  # Compte pour Location
    if "frais bancaires" in text.lower():
        return "627"  # Compte pour Frais bancaires
    if "impôts" in text.lower():
        return "63"   # Compte pour Impôts

    return libelle_to_account.get(libelle.split(" - ")[0], "62")


Traitement du corpus de texte et export dans un fichier csv

In [ ]:
# Traitement du corpus et extraction des transactions
def process_corpus(corpus):
    transactions = []
    for text in corpus:
        doc = nlp(text)
        dates = [ent.text for ent in doc.ents if ent.label_ == "DATE"]
        amounts = extract_amounts(text)
        libelle = determine_libelle(text)
        debit_account = get_debit_account(libelle, text)
        credit_account = get_credit_account(debit_account)
        for amount in amounts:
            date = dates[0] if dates else datetime.datetime.now().strftime("%Y-%m-%d")
            transactions.append({
                "date": date,
                "compte_debite": debit_account,
                "compte_credite": credit_account,
                "libelle": libelle,
                "montant_debit": amount,
                "montant_credit": amount
            })
    return transactions

In [ ]:
# Exemple de corpus de texte (à remplacer par le corpus réel)
corpus = texts
# Traiter le corpus
transactions = process_corpus(corpus)

# Créer un DataFrame
df = pd.DataFrame(transactions)

# Définir le chemin du fichier CSV
csv_file_path = 'journal_comptable.csv'

# Écrire dans le fichier CSV
df.to_csv(csv_file_path, mode='a', header=False, index=False)

print("Journal comptable mis à jour avec succès.")